# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Daniel Paniagua Ares  <br>
Url: https://github.com/dpaniaguaa/algoritmos_optimizacion_MIAR_2024<br>
Problema:
> 1. Sesiones de doblaje <br>

Descripción del problema:(copiar enunciado)
Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible.

Nº actores = 10

Nº tomas = 30

(*) La respuesta es obligatoria





                                        

In [1]:
import math
import numpy as np
import random
import copy
from itertools import combinations, chain

(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>

Si no hay ninguna restricción cada toma se puede asignar a uno de los días de grabación. Si no imponemos un límite en el número de tomas por día, podemos considerar que cada toma se puede grabar en un día independiente. Entonces, el número total de maneras en que se pueden organizar estas tomas es una permutación de 𝑛 tomas.

En este caso al contar con 30 tomas las posibilidades son







In [2]:
n_pos = math.factorial(30)

print(f"El número de posibilidades es: {n_pos}")

El número de posibilidades es: 265252859812191058636308480000000


¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.

Al tener en cuenta que sólo se pueden grabar 6 tomas por día eso reduce la manera de configruar cada día. Las posiblidades entonces son las posiblidades de cada día multiplicadas. Y cada día es una variación sin repetición por lo que:



In [3]:
n_pos = 1

for i in reversed(range(6,36,6)):
  n_pos = n_pos * (math.factorial(i)/(math.factorial(6)*math.factorial(i-6)))

print(f"El número de posibilidades es: {int(n_pos)}")

El número de posibilidades es: 1370874167589326336


Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


La mejor estructura es un array donde las columnas indiquen el actor y las filas la toma. De esta manera cada cela indica en qué toma es necesario qué actor/es. (Nótese que se ha hecho un array más pequeño para realizar pruebas sobre el algoritmo de fuerza bruta)

In [4]:
# A C T O R E S
# T
# O
# M
# A
# S

data = np.array([
        [1,1,1,1,1,0,0,0,0,0],
        [0,0,1,1,1,0,0,0,0,0],
        [0,1,0,0,1,0,1,0,0,0],
        [1,1,0,0,0,0,1,1,0,0],
        [0,1,0,1,0,0,0,1,0,0],
        [1,1,0,1,1,0,0,0,0,0],
        [1,1,0,1,1,0,0,0,0,0],
        [1,1,0,0,0,1,0,0,0,0],
        [1,1,0,1,0,0,0,0,0,0],
        [1,1,0,0,0,1,0,0,1,0],
        [1,1,1,0,1,0,0,1,0,0],
        [1,1,1,1,0,1,0,0,0,0],
        [1,0,0,1,1,0,0,0,0,0],
        [1,0,1,0,0,1,0,0,0,0],
        [1,1,0,0,0,0,1,0,0,0],
        [0,0,0,1,0,0,0,0,0,1],
        [1,0,1,0,0,0,0,0,0,0],
        [0,0,1,0,0,1,0,0,0,0],
        [1,0,1,0,0,0,0,0,0,0],
        [1,0,1,1,1,0,0,0,0,0],
        [0,0,0,0,0,1,0,1,0,0],
        [1,1,1,1,0,0,0,0,0,0],
        [1,0,1,0,0,0,0,0,0,0],
        [0,0,1,0,0,1,0,0,0,0],
        [1,1,0,1,0,0,0,0,0,1],
        [1,0,1,0,1,0,0,0,1,0],
        [0,0,0,1,1,0,0,0,0,0],
        [1,0,0,1,0,0,0,0,0,0],
        [1,0,0,0,1,1,0,0,0,0],
        [1,0,0,1,0,0,0,0,0,0],
        ])

data_small = np.array([
        [1,1,1,1,1,0,0,0,0,0],
        [0,0,1,1,1,0,0,0,0,0],
        [0,1,0,0,1,0,1,0,0,0],
        [1,1,0,0,0,0,1,1,0,0],
        [0,1,0,1,0,0,0,1,0,0],
        [1,1,0,1,1,0,0,0,0,0],
        [1,1,0,1,1,0,0,0,0,0],
        [1,1,0,0,0,1,0,0,0,0],
        [1,1,0,1,0,0,0,0,0,0],
        [1,1,0,0,0,1,0,0,1,0],
        [1,1,1,0,1,0,0,1,0,0],
        [1,1,1,1,0,1,0,0,0,0],

        ])

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

La función objetivo es el número de traslados de actores por día.

(*)¿Es un problema de maximización o minimización?

Es un problema de minimización, ya que lo que se busca es reducir el número de traslado de actores por día para pagar menos desplazamientos.

Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

In [5]:


# Función para calcular el número de actores necesarios para un conjunto de tomas
def actores_necesarios(tomas, data):
    if len(tomas) == 0:
        return 0
    return np.any(data[list(tomas)], axis=0).sum()

# Generar todas las combinaciones posibles de tomas por día
def generar_combinaciones(data, max_tomas_por_dia=6):
    indices = range(data.shape[0])
    todas_combinaciones = []
    for r in range(1, max_tomas_por_dia + 1):
        todas_combinaciones.extend(combinations(indices, r))
    return todas_combinaciones

# Evaluar todas las combinaciones posibles y encontrar la mejor
def encontrar_mejor_combinacion(data, n_tomas):
    mejor_combinacion = None
    min_actores = float('inf')
    todas_combinaciones = generar_combinaciones(data)
    
    # Generar todas las combinaciones posibles de grupos de tomas por día
    for combinacion in combinations(todas_combinaciones, (n_tomas + 5) // 6):
        # Verificar que todas las tomas estén cubiertas
        tomas_cubiertas = set(chain(*combinacion))
        if len(tomas_cubiertas) == n_tomas:
            actores = sum(actores_necesarios(tomas, data) for tomas in combinacion)
            
            if actores < min_actores:
                min_actores = actores
                mejor_combinacion = combinacion
    
    return mejor_combinacion, min_actores

# Número de tomas
n_tomas = data_small.shape[0]

mejor_combinacion, min_actores = encontrar_mejor_combinacion(data_small, n_tomas)
print("Mejor combinación de tomas por día:", mejor_combinacion)
print("Número mínimo de desplazamientos de actores necesarios:", min_actores)

Mejor combinación de tomas por día: ((0, 1, 2, 3, 4, 10), (5, 6, 7, 8, 9, 11))
Número mínimo de desplazamientos de actores necesarios: 14


Calcula la complejidad del algoritmo por fuerza bruta

La complejidad del algoritmo es cúbica ya que tiene 3 bucles anidados, el primero para iterar sobre las combinaciones, el segundo para iterar sobre las tomas dentro de las combinaciones, y el 3ro dentro de la función any de numpy que itera sobre el array.

o(n^3)

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

In [11]:
# Función para calcular el número de actores necesarios para un conjunto de tomas
def calcular_actores_necesarios(solucion,data):
    return sum(actores_necesarios(tomas, data) for tomas in solucion)

def tomas_cubiertas(solucion):
    # Verificar si una solución cubre todas las tomas
    tomas_cubiertas = set()
    
    for dia in solucion:
        tomas_cubiertas.update(dia)

    return len(tomas_cubiertas)


def actores_necesarios(tomas, data):
    if len(tomas) == 0:
        return 0
    return np.any(data[list(tomas)], axis=0).sum()


def generar_solucion_aleatoria(data):
  # Número total de tomas
  n_tomas = data.shape[0]

  # Crear una lista de todas las tomas
  todas_las_tomas = list(range(n_tomas))

  # Barajar la lista de tomas aleatoriamente
  random.shuffle(todas_las_tomas)

  # Dividir las tomas en grupos de hasta 6 tomas por día
  solucion = [todas_las_tomas[i:i+6] for i in range(0, n_tomas, 6)]

  return solucion


def generar_poblacion(data,N):
  return [generar_solucion_aleatoria(data) for _ in range(N)]

def evaluar_poblacion(poblacion):
    mejor_solucion = None
    min_actores = float('inf')
    max_tomas = 0
    
    for solucion in poblacion:
        tomas_cubiertas_ = tomas_cubiertas(solucion)
        if tomas_cubiertas_ > max_tomas:
            actores = calcular_actores_necesarios(solucion,data)
            if actores < min_actores:
                max_tomas = tomas_cubiertas_
                min_actores = actores
                mejor_solucion = solucion

    return (mejor_solucion, min_actores, max_tomas)


def crossover(sol1, sol2):
    # Punto de corte aleatorio
    punto_corte = random.randint(1, min(len(sol1), len(sol2)) - 1)
    
    # Generar hijos cruzando los padres en el punto de corte
    hijo1 = sol1[:punto_corte] + sol2[punto_corte:]
    hijo2 = sol2[:punto_corte] + sol1[punto_corte:]
    
    return hijo1, hijo2

def mutar(solucion, factor_mutacion, n_tomas):
    solucion_mutada = solucion.copy()
    
    for dia in solucion_mutada:
        if random.random() < factor_mutacion:
            # Seleccionar una toma aleatoria dentro del día
            toma_a_remover = random.choice(dia)
            dia.remove(toma_a_remover)
            
            # Agregar una nueva toma aleatoria que no esté ya en el día
            nueva_toma = random.choice(list(set(range(n_tomas)) - set(dia)))
            dia.append(nueva_toma)
    
    return solucion_mutada

def generar_nuevas_soluciones(sol1, sol2, factor_mutacion, n_tomas):
    # Generar soluciones cruzadas
    hijo1, hijo2 = crossover(sol1, sol2)
    
    # Aplicar mutación a los hijos
    hijo_mutado1 = mutar(hijo1, factor_mutacion, n_tomas)
    hijo_mutado2 = mutar(hijo2, factor_mutacion, n_tomas)
    
    return [hijo1, hijo2, hijo_mutado1, hijo_mutado2]


def cruzar(poblacion, mutacion):
  poblacion_copia = copy.deepcopy(poblacion)

  poblacion_final =  []

  while len(poblacion_copia) > 1:
    padre1,padre2  = random.sample(poblacion_copia,2)
    poblacion_copia.remove(padre1)
    poblacion_copia.remove(padre2)

    poblacion_final.extend(generar_nuevas_soluciones(padre1, padre2, mutacion, n_tomas))

 
  return poblacion_final

def seleccionar(poblacion,N, elitismo):
  poblacion_ordenada= sorted(poblacion, key=lambda sol: (-tomas_cubiertas(sol), calcular_actores_necesarios(sol, data)))

  n_elements = int(N*elitismo)

  nueva_poblacion = poblacion_ordenada[:n_elements]

  for i in range(n_elements, N, 1):
    nueva_poblacion.extend(random.sample(poblacion_ordenada,1))
  return nueva_poblacion

def algoritmo_genetico(data,N=100,mutacion=.15,elitismo=.1,generaciones=100):

  #Genera la poblacion inicial
  poblacion = generar_poblacion(data,N)

  #Inicializamos valores para la mejor solucion
  (solucion, n_actores, n_tomas_cubiertas) = evaluar_poblacion(poblacion)


  #Condicion de parada
  parar = False
  n=0
  #Inciamos el cliclo de generaciones
  while(parar == False) :

    #Cruce de la poblacion(incluye mutación)
    poblacion = cruzar(poblacion, mutacion)

    #Seleccionamos la población
    poblacion = seleccionar(poblacion, N, elitismo)

    #Evaluamos la nueva población
    (mejor_solucion, actores_min, tomas_cubiertas) = evaluar_poblacion(poblacion)
    
    if actores_min <= n_actores and tomas_cubiertas >= n_tomas_cubiertas:
        n_tomas_cubiertas = tomas_cubiertas
        n_actores = actores_min
        solucion = mejor_solucion

    print("Generacion #", n, "\nLa mejor solución es:" , mejor_solucion, "\ncon " , actores_min, " traslados de actores necesarios y cubre ",tomas_cubiertas, " tomas\n")

    #Numero de generaciones. Criterio de parada
    if n==generaciones:
      parar = True
    n +=1

  print("La mejor solucion encontrada ha sido: ",solucion)
  print("La cual necesita ", n_actores, " desplazamientos y cubre ", n_tomas_cubiertas," tomas")

  return solucion

algoritmo_genetico(data,N=1000,mutacion=0.2,elitismo=0.2,generaciones=100)

Generacion # 0 
La mejor solución es: [[19, 13, 11, 3, 21, 22], [9, 15, 1, 0, 14, 23], [17, 28, 26, 25, 7, 12], [10, 8, 6, 16, 18, 29], [27, 4, 19, 2, 13, 24]] 
con  39  traslados de actores necesarios y cubre  28  tomas

Generacion # 1 
La mejor solución es: [[9, 7, 27, 8, 1, 0], [17, 25, 19, 26, 22, 28], [5, 12, 23, 24, 16, 20], [2, 11, 3, 9, 14, 8], [29, 13, 6, 7, 18, 15]] 
con  37  traslados de actores necesarios y cubre  27  tomas

Generacion # 2 
La mejor solución es: [[1, 7, 3, 0, 24, 19], [25, 9, 29, 5, 11, 6], [27, 12, 18, 20, 2, 8], [13, 8, 28, 22, 26, 11], [15, 27, 21, 23, 24, 10]] 
con  38  traslados de actores necesarios y cubre  26  tomas

Generacion # 3 
La mejor solución es: [[14, 25, 5, 11, 28, 4], [26, 15, 16, 23, 5, 8], [12, 6, 13, 27, 10, 7], [20, 17, 2, 1, 4, 9], [16, 22, 18, 24, 29, 0]] 
con  38  traslados de actores necesarios y cubre  27  tomas

Generacion # 4 
La mejor solución es: [[26, 14, 15, 18, 3, 16], [28, 25, 11, 0, 23, 2], [12, 6, 13, 27, 10, 7], [20, 1

[[8, 18, 15, 17, 16, 1],
 [6, 20, 21, 28, 11, 9],
 [26, 29, 13, 7, 3, 14],
 [10, 24, 4, 12, 2, 19],
 [25, 0, 5, 27, 22, 23]]

(*)Calcula la complejidad del algoritmo

El mayor número de bucles anidados que nos encontramos dentro del algoritmo genético es de 2, por lo que su complejidad se queda en O(n^2)